In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%matplotlib inline

In [3]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), 
                                                     (0.3081))])

trainset = datasets.MNIST(root='dataset/', train=True, 
                                 transform=transform, download=True)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.MNIST(root='dataset/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)
classes=('zero','one','two','three','four','five','six','seven','eight','nine')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.leaky_relu(self.conv1(x)))
        x = self.pool(F.leaky_relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9,0.999))

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 160 == 159:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 160))
            running_loss = 0.0

print('Finished Training')

[1,   160] loss: 0.790


TypeError: ignored

In [ ]:
PATH = f'/content/gdrive/My Drive/Colab Notebooks/model2.pth'
torch.save(net.state_dict(), PATH)

In [7]:
PATH = f'/content/gdrive/My Drive/Colab Notebooks/model2.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [11]:
dataiter = iter(testloader)
images, labels = dataiter.next()[0].to(device),dataiter.next()[1].to(device)
print(images.size())
print(labels)
outputs = net(images)
print(outputs.size())
images, labels = dataiter.next()[0].to(device),dataiter.next()[1].to(device)
print(images.size())
print(labels)
outputs = net(images)
print(outputs.size())
# print images

torch.Size([64, 1, 28, 28])
tensor([7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
        6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7,
        3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5], device='cuda:0')
torch.Size([64, 10])
torch.Size([64, 1, 28, 28])
tensor([0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0,
        3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1,
        5, 9, 8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7], device='cuda:0')
torch.Size([64, 10])


In [10]:
outputs = net(images)
print(outputs.size())
print(outputs)

torch.Size([64, 10])
tensor([[-4.8482e+00, -4.0655e+00, -1.4446e+00,  1.1532e+00, -4.5406e+00,
          1.1153e-01, -6.7943e+00, -3.5455e+00,  1.4450e+01, -1.7311e+00],
        [-4.0298e+00, -5.1321e+00, -8.4444e+00,  1.4525e+00, -1.0047e+01,
          1.4975e+01, -4.4596e-01, -6.9609e+00,  4.5368e+00,  1.2091e+00],
        [-8.1792e-01, -5.7584e+00, -7.4624e+00, -5.1387e+00, -2.0894e+00,
          1.7404e+00,  1.0922e+01, -9.8311e+00,  3.0224e-01, -4.0424e+00],
        [ 7.9097e-01, -6.8932e+00, -6.9288e+00, -6.1405e+00, -4.2306e+00,
          2.0463e+00,  1.2413e+01, -1.0856e+01,  1.0318e+00, -6.1443e+00],
        [-5.0224e+00, -4.5462e+00, -7.0871e+00,  3.5734e+00, -1.0721e+01,
          1.4536e+01,  1.3720e+00, -7.3117e+00,  5.6851e+00, -2.0346e+00],
        [-5.2640e+00, -4.9199e+00,  4.7991e+00,  5.1784e+00, -1.4038e+00,
         -3.5279e+00, -1.9845e+01,  1.3017e+01,  1.2526e+00,  2.2067e+00],
        [-2.9762e+00, -3.1773e+00,  1.7041e+00,  2.0801e+00, -5.4062e+00,
         -1

In [10]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device),data[1].to(device)
        # calculate outputs by running images through the network 
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device),data[1].to(device)    
        outputs = net(images)    
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy for class zero  is: 99.2 %
Accuracy for class one   is: 99.5 %
Accuracy for class two   is: 98.8 %
Accuracy for class three is: 98.7 %
Accuracy for class four  is: 99.6 %
Accuracy for class five  is: 98.2 %
Accuracy for class six   is: 98.7 %
Accuracy for class seven is: 98.2 %
Accuracy for class eight is: 98.7 %
Accuracy for class nine  is: 96.0 %
